In [1]:
import sqlite3

### Ettevalmistus

In [2]:
# verbimustrite andmebaas, hetkel kasutan sealt ainult tabelit verb_patterns_len1
con = sqlite3.connect("verb_patterns_new.db")
cur = con.cursor()

In [3]:
# transaktsioonide andmebaasi lisamine (v32)
cur.execute('ATTACH DATABASE "v32_data.db" AS v32')

In [4]:
# uue andmebaasi lisamine, hetkel nimeks vp_data2. Siia salvestuvad loodavad tabelid
cur.execute('ATTACH DATABASE "vp_data2.db" AS vp')

### I tabel patterns

Veerud:

    pat_id - (mustri ID tabelis patterns_len1)
    pattern - (algne muster sõnena; NB! HETKEL ON SELLE VEERU READ MINGIL PÕHJUSEL NIHKES, SEETÕTTU PALUN HETKEL SEDA IGNOREERIDA, PARANDAN HILJEM)
    verb_word - (mustri (pea)verb)
    verb_compound - (pikema verbiühendi ülejäänud osad)
    phrase_nr - (fraasi number; kuna hetkel on vaatluse all ainult tabelist patterns_len1 pärit mustrid, on kõigil fraasidel number 1)
    phrase_case - (fraasi põhiliikme (pärast verbi) kääne; hiljem vaatame ilmselt vaid fraase, kus selles käändes on obliikva, kuid praeguseks pole seda tingimust veel sisse pandud)
    adp - (kaassõna)
    inf_verb - (infiniitverb)
    
Vajalik info saadakse tabelitest patterns_len1 ja transaction_head.
Hetkel on märgitud limiidiks 200, sest protseduur on aeganõudev. See tähendab, et tabelis patterns on tabelist patterns_len1 esimesed 200 mustrit, millel on lisaks põhiverbile kuni üks verbiühendi osa ning millel ei ole tabelis patterns_len1 *other*-kategooriasse kuuluvat mustriliiget.
Verbiühendeid, millel on lisaks põhiverbile rohkem, kui üks osa, on transaktsioonide andmebaasis võrdlemisi vähe.
*Other*-kategooria lahendamine on natuke keerukas (sidesõnad, sõnad nagu 'millal', 'kuidas' jne).

In [5]:
cur.execute("""
DROP TABLE IF EXISTS vp.patterns
""")

cur.execute("""
CREATE TABLE vp.patterns AS
SELECT DISTINCT
    pat.ID AS pat_id,
    pat.word || ' ' || pat.government AS pattern,
    pat.verb_word AS verb_word,
    pat.compound_prt1 AS verb_compound,
    pat.phrase_nr AS phrase_nr,
    pat.w_case AS phrase_case,
    pat.adp AS adp,
    pat.verb AS inf_verb
FROM 
    patterns_len1 as pat
INNER JOIN 
    v32.transaction_head
ON 
    pat.verb_word = v32.transaction_head.verb
WHERE 
    pat.compound_prt1 = v32.transaction_head.verb_compound
AND 
    pat.compound_prt2 = ''
AND 
    pat.compound_prt3 = ''
AND 
    pat.other = ''
""")

cur.execute("""
CREATE INDEX vp.pat_id_idx ON patterns(pat_id)
"""
)

cur.execute("""
CREATE INDEX vp.phrase_case_idx ON patterns(phrase_case)
"""
)

cur.execute("""
CREATE INDEX vp.adp_idx ON patterns(adp)
"""
)

cur.execute("""
CREATE INDEX vp.inf_verb_idx ON patterns(inf_verb)
"""
)

cur.execute("""
CREATE INDEX vp.verb_word_idx ON patterns(verb_word)
"""
)

cur.execute("""
CREATE INDEX vp.verb_compound_idx ON patterns(verb_compound)
"""
)

cur.execute("""
CREATE INDEX vp.phrase_nr_idx ON patterns(phrase_nr)
"""
)

### Abitabel asjade kättesaamiseks

Mitte-elegantne viis saada kätte kõik **head_id**-d, millele vastavates fraasides on esindatud kõik vaadeldavate mustrite osised (sobiv kääne (kui on), kaassõna (kui on), infiniitverb (kui on)). Saab kasutada ülejäänud tabelite koostamiseks. Ilmselt on võimalik teha tegelikult ära ka JOIN-ide abil. Limiidiks on 500, sest tegemist on taaskord üsna aeganõudva protsessiga.

In [6]:
cur.execute("""
DROP TABLE IF EXISTS vp.verb_phrase_matches_step1
""")
cur.execute("""
DROP TABLE IF EXISTS vp.verb_phrase_matches_step2
""")
cur.execute("""
DROP TABLE IF EXISTS vp.verb_phrase_matches_step3
""")

cur.execute("""
CREATE TABLE vp.verb_phrase_matches_step1 AS
SELECT pat_id,
    tr.head_id,
    phrase_case,
    adp,
    inf_verb,
    phrase_nr
FROM 
(
    SELECT pat.pat_id as pat_id,
        tr_head.id as head_id,
        pat.phrase_case as phrase_case,
        pat.adp as adp,
        pat.inf_verb as inf_verb,
        pat.phrase_nr as phrase_nr
    FROM 
        vp.patterns as pat
    INNER JOIN 
        v32.transaction_head as tr_head
    ON
        pat.verb_word=tr_head.verb
    WHERE
        pat.verb_compound=tr_head.verb_compound
) as pat_tr_joined
INNER JOIN 
    v32.`transaction` as tr
ON
    pat_tr_joined.head_id=tr.head_id
WHERE 
    pat_tr_joined.phrase_case = '' OR instr(tr.feats, pat_tr_joined.phrase_case) > 0
""")


cur.execute("""
CREATE TABLE vp.verb_phrase_matches_step2 AS
SELECT 
    pat_id,
    tr.head_id,
    phrase_case,
    adp,
    inf_verb,
    phrase_nr
FROM 
    vp.verb_phrase_matches_step1 as step1
INNER JOIN 
    v32.`transaction` as tr
ON 
    step1.head_id=tr.head_id
WHERE 
    step1.adp = '' OR (tr.form = step1.adp AND tr.deprel = 'case')
""")


cur.execute("""
CREATE TABLE vp.verb_phrase_matches_step3 AS
SELECT pat_id,
    tr.head_id,
    phrase_case,
    adp,
    inf_verb, 
    phrase_nr
FROM 
    vp.verb_phrase_matches_step2 as step2
INNER JOIN
    v32.`transaction` as tr
ON 
    step2.head_id=tr.head_id
WHERE 
    step2.inf_verb = '' OR (tr.form = step2.inf_verb AND (instr(tr.feats, 'inf') > 0 OR instr(tr.feats, 'sup') > 0))
""")


cur.execute("""
DROP TABLE IF EXISTS vp.verb_phrase_matches_step1
""")
cur.execute("""
DROP TABLE IF EXISTS vp.verb_phrase_matches_step2
""")

### II tabel patterns_meta

Veerud:

    pat_id - mustri ID
    phrase_count - mustrile vastavate fraaside (limiteeritud) hulk transaktsioonide andmebaasis
    
Asjade kättesaamiseks kasutan abitabelit, sealt leian esiteks unikaalsed read, mis seejärel kokku loetakse. Tuleb endiselt silmas pidada, et vastete hulk transaktsioonide andmebaasist on hetkel piiratud.

In [7]:
cur.execute("""
DROP TABLE IF EXISTS vp.patterns_meta
""")

cur.execute("""
CREATE TABLE vp.patterns_meta (
    pat_id INTEGER,
    phrase_count INTEGER
)
""")

cur.execute("""
INSERT INTO vp.patterns_meta(
    pat_id,
    phrase_count
)
SELECT 
    pat_id,
    count(*) AS phrase_count
FROM
(
    SELECT DISTINCT
        pat_id, 
        head_id
    FROM
        vp.verb_phrase_matches_step3
) AS tbl
GROUP BY
    tbl.pat_id
ORDER BY
    phrase_count DESC
""")

cur.execute("""
CREATE INDEX vp.meta_pat_id_idx ON patterns_meta(pat_id)
"""
)

con.commit()

### III tabel verb_phrase_matches

Veerud:

    pat_id - mustri ID
    head_id - verbi ID transaktsioonide andmebaasis
    phrase_nr - fraasi nr
    
Asjade kättesaamiseks kasutan abitabelit, sealt leian unikaalsed read. Tuleb endiselt silmas pidada, et vastete hulk transaktsioonide andmebaasist on hetkel piiratud.
Kui tulevikus võtta juurde pikemad mustrid, mis sisaldavad rohkem, kui ühte fraasi, siis saab esimene fraas olema eristatud tähistatud numbriga 1 ning teine numbriga 2.

In [8]:
cur.execute("""
DROP TABLE IF EXISTS vp.verb_phrase_matches
""")

cur.execute("""
CREATE TABLE vp.verb_phrase_matches (
    pat_id INTEGER,
    head_id INTEGER,
    phrase_nr INTEGER
)
""")

cur.execute("""
INSERT INTO vp.verb_phrase_matches(
    pat_id,
    head_id,
    phrase_nr
)
SELECT DISTINCT
    pat_id,
    head_id,
    phrase_nr
FROM
    vp.verb_phrase_matches_step3
""")

cur.execute("""
CREATE INDEX vp.match_pat_id_idx ON verb_phrase_matches(pat_id)
"""
)

cur.execute("""
CREATE INDEX vp.match_head_id_idx ON verb_phrase_matches(head_id)
"""
)

cur.execute("""
CREATE INDEX vp.match_phrase_nr_idx ON verb_phrase_matches(phrase_nr)
"""
)


con.commit()

### IV tabel verb_matches

Veerud:

    pat_id - mustri ID
    head_id - verbi ID transaktsioonide andmebaasis
    
Asjade kättesaamiseks kasutan tabelit patterns, et saada kätte (piiratud hulga) mustrite verbid ning transaktsioonide tabelit transaction_head, et saada kätte viited kõigile lausetele, kus verbid esinevad. Ülejäänud mustrit pole selle tabeli puhul arvesse võetud.

In [9]:
# tabel verb_matches
cur.execute("""
DROP TABLE IF EXISTS vp.verb_matches
""")

cur.execute("""
CREATE TABLE vp.verb_matches (
    pat_id INTEGER,
    head_id INTEGER
)
""")

cur.execute("""
INSERT INTO vp.verb_matches(
    pat_id,
    head_id
)
SELECT DISTINCT
    pat_id,
    head_id
FROM
(
    SELECT pat_id, tr_head.id as head_id
    FROM
        patterns as pat
    INNER JOIN
        v32.transaction_head as tr_head
    ON
        pat.verb_word = tr_head.verb
    WHERE pat.verb_compound = tr_head.verb_compound
)
""")

cur.execute("""
CREATE INDEX vp.v_match_pat_id_idx ON verb_matches(pat_id)
"""
)

cur.execute("""
CREATE INDEX vp.v_match_head_id_idx ON verb_matches(head_id)
"""
)


con.commit()

In [ ]:
# soovi korral saab kustutada abitabeli, aga SQLITE-s see tegevus mäluruumi ei vabasta

#cur.execute("""
#DROP TABLE IF EXISTS vp.verb_phrase_matches_step3
#""")

In [10]:
# ühenduse sulgemine
con.close()